### MongoDB Atlas 내담자정보 similarity test(ANN)

In [24]:
from pymongo import MongoClient
from openai import OpenAI
from pathlib import Path
from dotenv import load_dotenv
import os

# 환경변수 설정
env_path = Path('..') / '.env'
load_dotenv(env_path)

# OpenAI & MongoDB 클라이언트 설정
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
mongo_client = MongoClient(os.getenv('MONGODB_ATLAS_CLUSTER_URI'))
db = mongo_client['HelloWorld-AI']
collection = db['foreigner_legal_test']

In [ ]:
def search_similar_cases(query_text, limit=3):
    try:
        # 쿼리 텍스트 임베딩
        query_embedding = client.embeddings.create(
            model="text-embedding-3-large",
            input=query_text
        ).data[0].embedding

        # ANN Vector Search 쿼리
        results = collection.aggregate([
            {
                "$vectorSearch": {
                    "index": "vector_index",
                    "path": "내담자_정보.Embedding",
                    "queryVector": query_embedding,
                    "numCandidates": 200,  # ANN 검색에 필요
                    "limit": limit
                }
            },
            {
                "$project": {
                    "내담자_정보": 1,
                    "해결방법": 1, #해결방법 필드도 반환
                    "score": { "$meta": "vectorSearchScore" },
                    "_id": 0
                }
            }
        ])

        print(f"\n'{query_text}'와(과) 가장 유사한 {limit}개의 사례:\n")
        
        results_list = list(results)
        if not results_list:
            print("검색 결과가 없습니다.")
            return

        for idx, result in enumerate(results_list, 1):
            print(f"=== 유사 사례 {idx} ===")
            print(f"유사도 점수: {result.get('score', 'N/A')}")
            info = result['내담자_정보']
            print(f"거주지역: {info.get('거주지역', 'N/A')}")
            print(f"국적: {info.get('국적', 'N/A')}")
            print(f"체류자격: {info.get('체류자격', 'N/A')}")
            print(f"추가정보: {info.get('추가정보', 'N/A')}")

            # 해결방법 출력.. 개행 추가해야됨
            print("\n[해결방법]")
            print(result.get('해결방법', 'N/A'))
            print()

    except Exception as e:
        print(f"검색 중 에러 발생: {str(e)}")

In [32]:
# 사용 예시
if __name__ == "__main__":
    # 검색 쿼리 예시들
    queries = [
        "체류자격 만료된 외국인 근로자",
        "중도 퇴사한 우즈베키스탄 근로자",
        "E-9 비자 문제"
    ]
    
    # 사용자 입력 받기
    print("검색하고 싶은 내용을 입력하세요: ")
    user_query = input().strip()
    
    # 검색 실행
    search_similar_cases(user_query, limit=3)

검색하고 싶은 내용을 입력하세요: 

''체류자격 만료까지 얼마 안 남았습니다.''와(과) 가장 유사한 3개의 사례:

=== 유사 사례 1 ===
유사도 점수: 0.7408804893493652
거주지역: 양주시
국적: 태국
체류자격: E-9
추가정보: 
제조 공장 근무 중
약 4년 10개월간 근로계약 체결
최근 체류자격 박탈 문제 발생
고용 허가 기간 연장 신청 미처리로 인한 체류 기간 초과
휴대폰 정지로 체류기간 초과 사실 확인


[해결방법]


사업장 담당자와 함께 방문하여 상황을 정리하고, 고용허가기간 연장 신청 누락을 확인한 후 변경 신청서를 권익보호협의회에 제출하였습니다. 관련 자료를 수집하고, 의정부고용센터에 증빙자료를 제출한 결과, 권익보호협의회 심의를 통해 노동자의 사업장변경 신청이 허가되었습니다. 이를 바탕으로 근로자는 체류 자격을 다시 확보할 수 있었습니다.


=== 유사 사례 2 ===
유사도 점수: 0.7333495616912842
거주지역: 시흥시
국적: 필리핀
체류자격: E-9
추가정보: 
한국 체류 기간이 1년 4개월 남은 상태
제2회사를 포함하여 여러 면접에서 채용 기피 경험
쉼터 입소 요청했으나 자리 부족으로 자취방 계약하여 거주 중
고시원 거주 후 구직활동, 생활비 부족으로 쉼터 입소 원함
합법적인 구직 기간이 7일 남아 구직 실패시 출국 위기


[해결방법]


노동부 알선 회사를 통해 여러 번 면접을 봤으나 채용이 이루어지지 않음. 마지막 구직 가능일에 노동부 담당자의 도움을 받아 남양주 지역 가구 회사에 취업 확정. 이후 근로계약을 체결하고 쉼터에서 퇴소하여 회사 기숙사로 이동. 남은 짐은 토요일에 가져옴. 한국에서 남은 1년 4개월 동안 근무 가능하며, 이후 성실 근로자로서의 연장을 희망. 현재는 회사에 잘 적응하며 근무 중으로, 내년에 근로계약 만료 전에 성실근로자 신청 예정.


=== 유사 사례 3 ===
유사도 점수: 0.7323595881462097
거주지역: 남양주시
국적: 방글라데시
체류자격: 미등록


In [ ]:
user_query = """
"거주지역": "안산시",
"국적": "우즈베키스탄",
"체류자격": "E-9",
"추가정보": "3개월 동안 임금 체불, 회사 대표와 협상 불가"
}
"""

# 검색 실행
search_similar_cases(user_query, limit=3)


'
"거주지역": "안산시",
"국적": "우즈베키스탄",
"체류자격": "E-9",
"추가정보": "3개월 동안 임금 체불, 회사 대표와 협상 불가"
}
'와(과) 가장 유사한 3개의 사례:

=== 유사 사례 1 ===
유사도 점수: 0.8515961170196533
거주지역: 여주시
국적: 우즈베키스탄
체류자격: E-9
추가정보: 
중도 퇴직한 외국인 근로자
2020년 종합소득세 체납으로 고용허가 연장 불가능했던 상황
현재까지 체류자격 연장이 되지 않아 불안함
체류자격은 당장 곧 만료 예정


[해결방법]


체납된 세금에 대한 원인을 파악하고 체납 금액을 납부하였습니다. 2020년에 중도 퇴직 후 근로소득 신고를 하지 않아 체납되었던 사항을 해결하기 위해, 관련 법령을 안내받고 필요한 서류를 제출하였습니다. 출입국사무소 방문 예약과 고용허가 발급 절차를 확인하여 체류자격 연장을 완료하였습니다. 또한, 사회통합프로그램 사전평가 신청을 통해 장기적으로 안정된 체류가 가능하도록 지원받았습니다.

=== 유사 사례 2 ===
유사도 점수: 0.8502938747406006
거주지역: 안산시
국적: 우즈베키스탄
체류자격: F-4
추가정보: 
건설현장에서 근무한 이주 노동자 이OOO
6월부터 임금 460만원 체불
건설현장에서 일당 20만원으로 23일 근무
임금체불 진정서를 안산고용노동지청에 접수
임금을 지급받지 못해 어려움을 겪음
사업주는 본국으로 잠시 떠남


[해결방법]


이주노동자 이OOO는 체불된 임금 문제로 인해 안산고용노동지청에 임금체불 진정서를 접수했습니다. 그러나 사업주의 조카인 피진정인이 임금 액수에 대해 이견을 보여, 근로감독관이 입증 가능한 자료가 없다는 이유로 해결에 어려움이 있었습니다. 8월 15일까지 약속된 금액이 지급되지 않으면 추가 조치를 취할 계획입니다. 진정인이 피진정인과의 대화 내용을 녹취한 상태이며, 필요한 경우 근로감독관에게 해당 녹취를 제공할 예정입니다. 건설업계에서 구두로 근로계약을 체결하는 문제가 반복되고 있으며, 언

In [36]:
user_query = """
한국인 남성과 결혼했는데, 가정폭력에 시달리고 있습니다.
"""

# 검색 실행
search_similar_cases(user_query, limit=3)


'
한국인 남성과 결혼했는데, 가정폭력에 시달리고 있습니다.
'와(과) 가장 유사한 3개의 사례:

=== 유사 사례 1 ===
유사도 점수: 0.7470093369483948
거주지역: 안산시
국적: 기타
체류자격: F-1
추가정보: 
최근 입국
슬라브계 여성
F-1 비자 소지자
미성년 자녀 3명 양육 중
경제활동이 어려움
남편의 폭력으로 접근금지 명령


[해결방법]


남편의 폭력 신고 후 접근금지 명령을 받았고, (사)너머 지원단체의 도움을 통해 생필품, 유아용품, 긴급의료비 및 월세를 지원받았습니다. 적십자사로부터 긴급의료비도 지원받아 치료를 완료했고, 현재 아르바이트를 시작하여 일부 경제활동을 하며 밀린 공과금을 납부했습니다. 아동보호전문기관의 사례관리 상담을 받으며 자녀들의 상태를 점검하고, 통역 및 자녀 교육 지침도 지원되었습니다. 내국인들의 지원으로 시급한 문제들은 해결되었으나 향후 자녀 양육비와 함께 한국에서의 자립을 위한 지속적인 지원과 관심이 필요한 상황입니다.


=== 유사 사례 2 ===
유사도 점수: 0.736481249332428
거주지역: 성남시
국적: 베트남
체류자격: 미등록
추가정보: 
베트남 출신 이주 노동자
약 10년간 한국 체류 중 미등록 상태
한국 남자친구와 약 2년간 동거
남자친구는 결혼 상태였으며 시어머니의 반대
아이 출산 후 혼인신고 및 아이의 호적 등록 미완료
남자친구는 신용불량 상태로 경제적 어려움
변호사를 통한 문제 해결 시도 후 결론을 못 내림
20개월 아들과 함께 베트남으로 귀국 결정


[해결방법]


기관의 지원을 통해 생필품 제공 및 상담 진행
남자친구와의 상담을 통해 경제적 및 법적 문제 파악 시도
변호사와의 상담을 통해 국제결혼 및 법적 문제를 해결하려 했으나, 남자친구의 상황으로 인해 어려움 존재
내담자는 닥친 문제들이 해결될 가능성이 낮다고 판단, 더 이상 한국에 머무를 수 없음을 느끼고 귀국 결정
귀국 전까지 친언니의 도움을 받으며 지냄
베트남 귀국 후에도 아이를 양육하며 새로운 시작을 